In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch
import numpy as np
from tqdm import trange
import random
from torch.utils.data import TensorDataset
import matplotlib.pyplot as plt
import warnings 
import pandas as pd
torch.manual_seed(0)
np.random.seed(0)
torch.cuda.manual_seed_all(0)
random.seed(0)

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
warnings.filterwarnings("ignore")

In [3]:
mask_mono = np.array([-1, 1, -1, -1, 1] + [0] * 23)

In [4]:
def split_input(inputs):
    return inputs[:, np.where(mask_mono==0)].squeeze(), \
        inputs[:, np.where(mask_mono!=0)].squeeze() * torch.tensor(mask_mono[np.where(mask_mono!=0)][None,:], dtype=torch.float32).to(device)

In [5]:
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=50):
    losses = []
    val_losses = []
    best_val_acc = 0.
    for _ in trange(num_epochs):
        model.train()
        total = 0
        losses_buffer = []
        for inputs, labels in train_loader:
            inputs_free, inputs_mono = split_input(inputs)
            optimizer.zero_grad()
            outputs = model(inputs_free.float(), inputs_mono.float())
            loss = criterion(outputs, labels.float())
            losses_buffer.append(loss)
            loss.backward()
            optimizer.step()
            
            total += labels.size(0)
        losses.append(np.mean([el.detach().cpu() for el in losses_buffer]))
        
        val_acc, val_loss = validate_model(model, val_loader, criterion)
        best_val_acc = max(best_val_acc, val_acc)
        val_losses.append(val_loss)
    
    return losses, val_losses, best_val_acc

def validate_model(model, val_loader, criterion):
    model.eval()
    val_loss = []
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs_free, inputs_mono = split_input(inputs)
            outputs = model(inputs_free.float(), inputs_mono.float())
            loss = criterion(outputs, labels.float())
            val_loss += [loss.item()]

            total += labels.size(0)
            correct += (outputs.round() == labels).sum().item()

    val_accuracy = correct / total
    return val_accuracy, np.mean(val_loss)

In [6]:
class MonotonicLinear(nn.Linear):
    def __init__(
        self,
        in_features: int, 
        out_features: int, 
        bias: bool = True,
        device=None, 
        dtype=None,
        pre_activation=nn.Identity(),
    ):
        super().__init__(in_features, out_features, bias=bias, device=device, dtype=dtype)
        self.act = pre_activation
        
    def forward(self, x):
        w_pos = self.weight.clamp(min=0.0)
        w_neg = self.weight.clamp(max=0.0)
        x_pos = F.linear(self.act(x), w_pos, self.bias)
        x_neg = F.linear(self.act(-x), w_neg, self.bias)  
        return x_pos + x_neg
    
class MonoModel(torch.nn.Module):
    def __init__(self, input_size_mono, num_layers_mono, num_layers_pre_mono, num_neurons_mono, num_neurons_pre_mono, activation=nn.ReLU()) -> None:
        super().__init__()
        self.pre_mono = torch.nn.ModuleList([torch.nn.LazyLinear(num_neurons_pre_mono) for _ in range(num_layers_pre_mono)])
        self.mono = torch.nn.ModuleList(
            [
                MonotonicLinear(input_size_mono + num_neurons_pre_mono, num_neurons_mono, pre_activation=nn.Identity()),
                *[MonotonicLinear(num_neurons_mono, num_neurons_mono, pre_activation=activation) for _ in range(num_layers_mono)],
                MonotonicLinear(num_neurons_mono, 1, pre_activation=activation),
            ]
        )
    def forward(self, x, x_mono):
        for layer in self.pre_mono:
            x = torch.nn.functional.relu(layer(x))
        
        x = torch.cat((x, x_mono), dim=-1)
        for layer in self.mono:
            x = layer(x)
        
        return x
        

In [7]:
def run(SEED, n, lr=1e-3, activation=nn.ReLU()):
    torch.manual_seed(SEED)
    np.random.seed(SEED)
    torch.cuda.manual_seed_all(SEED)
    random.seed(SEED)

    df_train = pd.read_csv('data/train_loan.csv',header = None)
    df_train = df_train.dropna(axis=0)        
    X_train = df_train.to_numpy()[:,:-1]
    y_train = df_train.to_numpy()[:,-1:]

    df_val = pd.read_csv('data/test_loan.csv',header = None)
    df_val = df_val.dropna(axis=0)
    X_val = df_val.to_numpy()[:,:-1]
    y_val = df_val.to_numpy()[:,-1:]

    X_train = torch.tensor(X_train).to(device).float()
    X_val = torch.tensor(X_val).to(device).float()
    y_train = torch.tensor(y_train).to(device).float()
    y_val = torch.tensor(y_val).to(device).float()

    train_loader = torch.utils.data.DataLoader(TensorDataset(X_train, y_train), batch_size=256, shuffle=True, drop_last=True)
    val_loader = torch.utils.data.DataLoader(TensorDataset(X_val, y_val), batch_size=256, shuffle=True, drop_last=True)
    
    model = MonoModel((mask_mono!=0).sum(),3,3, n,n, activation=activation).to(device)
    criterion = torch.nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr)
    losses, val_losses, best_val_acc = train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=50)
    print("SEED", SEED, "VAL LOSS", np.min(val_losses), "BEST ACC", best_val_acc)
    return losses, val_losses, best_val_acc

In [8]:
n = 16
activation = nn.ReLU()
train_losses, val_losses, val_acc = [],[],[]
for seed in range(5):
    ltrain, lval, lacc = run(seed, n, activation=activation)
    train_losses.append(ltrain)
    val_losses.append(lval)
    val_acc.append(lacc)

print("---------------------------------")
print("Mean acc", np.mean(val_acc))
print("Std acc", np.std(val_acc))
print("---------------------------------")

100%|██████████| 50/50 [08:09<00:00,  9.79s/it]


SEED 0 VAL LOSS 0.21636439011479816 BEST ACC 0.6531421076642335


100%|██████████| 50/50 [07:33<00:00,  9.06s/it]


SEED 1 VAL LOSS 0.21629490621768646 BEST ACC 0.6540830291970803


100%|██████████| 50/50 [06:30<00:00,  7.82s/it]


SEED 2 VAL LOSS 0.21638492973398987 BEST ACC 0.6533559534671532


100%|██████████| 50/50 [06:34<00:00,  7.90s/it]


SEED 3 VAL LOSS 0.21633010156398272 BEST ACC 0.6535840556569343


100%|██████████| 50/50 [06:02<00:00,  7.25s/it]

SEED 4 VAL LOSS 0.21632703529657238 BEST ACC 0.6540545164233577
---------------------------------
Mean acc 0.6536439324817518
Std acc 0.00037409367623457104
---------------------------------
